## **Agent Memory with Redis**

### **I. Introduction**

Without memory, AI agents are like goldfish - they forget everything after each conversation and can't learn from past interactions or maintain context across sessions. Agentic systems require both **short-term** and **long-term** memory in order to complete tasks in a personalized and resilient manner. Memory is all about state management and `Redis` is the well-known in-memory database for exactly this kind of use case today in production systems.

### **II. What we'll build**

This tutorial demonstrates how to build a **memory-enabled travel agent** with **Redis and LangGraph** that remembers user preferences and provides personalized recommendations. This is a horizontal concept that you can take and apply to your own agent use cases.

We'll explore:
1. Short-term memory management using LangGraph's checkpointer
2. Long-term memory storage and retrieval using RedisVL
3. Managing long-term memory as a tool for a ReAct agent
4. Managing conversation history size with summarization

### **III. Memory architecture overview**

Ouer agent uses a dual-memory system:
* **Short-term**: Manages conversation context
* **Long-term**: Stores persistent knowledge

#### **1. Short-term Memory**

The agent tracks chat history using Redis through LangGraph's checkpointer. Each node in the graph (Retrieve Memories, Respond, Summarize) saves its state to Redis, including conversation history and thread metadata.

<img src="short-term-memory.png" alt="alt text" width="400px" height="400px" style="background-color: lightblue;"/>

To prevent context window pollution, the agent summarizes conversations when they exceed a configurable length.

#### **2. Long-term Memory**

Long-term memories are stored & indexed in Redis using the RedisVL client, with 2 types:
* **Episodic**: User preferences and experiences
* **Semantic**: General travel knowledge

<img src="long-term-memory.png" alt="alt text" width="500px" height="300px" style="background-color: lightblue;"/>

### **IV. Set up enviroment**

In [1]:
!uv add langchain-google-genai langgraph-checkpoint langgraph langgraph-checkpoint-redis langchain-redis

Resolved 145 packages in 17ms
Audited 142 packages in 39ms


#### **1. Required API keys**

In [2]:
import getpass
import os

def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"Enter your {key}: ")

_set_env("GOOGLE_API_KEY")

#### **2. Setup Redis**

In [6]:
import os
from dotenv import load_dotenv
from redis import Redis

load_dotenv()

redis_host = os.getenv("REDIS_HOST")
redis_port = os.getenv("REDIS_PORT")
redis_username = os.getenv("REDIS_USERNAME")
redis_password = os.getenv("REDIS_PASSWORD")

redis_client = Redis(
    host = redis_host,
    port = redis_port,
    decode_responses = True,
    username = redis_username,
    password = redis_password
)

redis_client.ping()

True

#### **3. Prepare memory data models**

In this section, we'll create a robust data modeling system for our agent's memory using `Pydantic`. These models will ensure type safety and provide clear data strutures for storing and retrieving memories form Redis.

We'll implement 4 key components:
1. `MemoryType` - An enumeration that categorizes memories into 2 types:
* Episodic: Personal experiences and user preferences
* Semantic: General knowledge and domain facts
2. `Memory` - The core model representing a single memory entry with its content and metadata
3. `Memories` - A container model that holds collections of memory objects
4. `StoredMemory` - A specialized model for memories that have been persistend to Redis

These models work together to create a complete memory lifecycle, from creation to storage and retrieval.

In [7]:
import ulid

from datetime import datetime
from enum import Enum
from typing import List, Optional
from pydantic import BaseModel, Field

In [8]:
class MemoryType(str, Enum):
    """
    Defines the type of long-term memory for categorization and retrieval.

    EPISODIC: Personal experiences and user-specific preferences
            (e.g., "User prefers Delta airlines", "User visited Paris last year")
    
    SEMANTIC: General domain knowledge and facts
            (e.g., "Singapore requires passport", "Tokyo has excellent public transit")

    The type of a long-term memory.

    EPISODIC: User specific experiences and preferences

    SEMANTIC: General on top of the user's preferences and LLM's training data.
    """

    EPISODIC = "episodic"
    SEMANTIC = "semantic"

In [9]:
class Memory(BaseModel):
    """Represents a single long-term memory."""

    content: str
    memory_type: MemoryType
    metadata: str

In [10]:
class Memories(BaseModel):
    """A list of memories extracted from a conversation by an LLM."""

    memories: List[Memory]

In [11]:
class StoredMemory(Memory):
    """A stored long-term memory"""

    id: str # The redis key
    memory_id: ulid.ULID = Field(default_factory=lambda: ulid.ULID())
    created_at: datetime = Field(default_factory=datetime.now)
    user_id: Optional[str] = None
    thread_id: Optional[str] = None
    memory_type: Optional[MemoryType] = None

Now we have type-safe data models that handle the complete memory lifecycle from LLM extraction to Redis storage, with proper metadata tracking for production use. Next, we'll set up the Redis infrastructure to store and search these memories using vector embeddings.

### **V. Memory Storage**

- **Short-term memory** is handled automatically by `RedisSaver` from `langgraph-checkpoint-redis`
- **Long-term memory**, we'll use RedisVL with vector embeddings to enable semantic search of past experiences and knowledge.

Below, we'll create a search index schema in Redis to hold our long term memories. The schema has a few different fields including content, memory type, metadata, timestamps, user id, memory id, and the embedding of the memory.

In [12]:
from redisvl.index import SearchIndex
from redisvl.schema.schema import IndexSchema

# Define the schema for our vector search index
# This creates the structure for storing and query memories
memory_schema = IndexSchema.from_dict({
    "index": {
        "name": "agent_memories", # Index name for identification
        "prefix": "memory", # Redis key prefix (memory:1, memory:2, ...)
        "key_separator": ":",
        "storage_type": "json"
    },
    "fields": [
        {"name": "content", "type": "text"},
        {"name": "memory_type", "type": "tag"},
        {"name": "metadata", "type": "text"},
        {"name": "created_at", "type": "text"},
        {"name": "user_id", "type": "tag"},
        {"name": "memory_id", "type": "tag"},
        {
            "name": "embedding", 
            "type": "vector",
            "attrs": {
                "algorithm": "flat",
                "dims": 768,
                "distance_metric": "cosine",
                "datatype": "float32"
            }
        }
    ]
})

Below we create the `SearchIndex` from the `IndexSchema` and our Redis client connection object. We'll overwrite the index spec if its already created!

In [13]:
try:
    long_term_memory_index = SearchIndex(
        schema = memory_schema,
        redis_client = redis_client,
        validate_on_load = True
    )

    long_term_memory_index.create(overwrite = True)
    print("Long-term memory index ready")
except Exception as e:
    print(f"Error creating index: {e}")

Long-term memory index ready


#### **1. Functions to access memories**

Next, we provide 3 core functions to access, store and retrieve memories. We'll eventually use these in tools for the LLM to call. We'll start by loading a vectorizer class to create GEMINI embeddings.

In [14]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

gg_embed = GoogleGenerativeAIEmbeddings(
    model = 'text-embedding-004',
    api_key = os.environ["GOOGLE_API_KEY"]
)

Next we'll set up a simple logger so our functions will record log activity of what's happening.

In [15]:
import logging

logger = logging.getLogger(__name__)

##### **a. Check for similar memories**

First, we'll write a utility function to check if a memory similar to a given memory already exists in the index.

This function checks for duplicate memories in Redis by:
1. Converting the input content into a vector embedding
2. Creating filters for user_id and memory_type
3. Using vector similarity search with a vector range query to find any existing + similar memories
4. Returning True if a similar memory exists, False otherwise

This helps prevent storing redundant information in the agent's memory.

In [16]:
from redisvl.query import VectorRangeQuery
from redisvl.query.filter import Tag

# If we have any memories that aren't associated with a user, we'll use this ID
SYSTEM_USER_ID = "system"

def similar_memory_exists(
    content: str,
    memory_type: MemoryType,
    user_id: str = SYSTEM_USER_ID,
    thread_id: Optional[str] = None,
    distance_threshold: float = 0.1,
) -> bool:
    """Check if a similar long-term memory already exists in Redis"""
    content_embedding = gg_embed.embed_documents([content])
    content_embedding = content_embedding[0]

    filters = (Tag("user_id") == user_id) & (Tag("memory_type") == memory_type)

    if thread_id:
        filters = filters & (Tag("thread_id") == thread_id)

    vector_query = VectorRangeQuery(
        vector = content_embedding,
        num_results = 1,
        vector_field_name = "embedding",
        filter_expression = filters,
        distance_threshold = distance_threshold,
        return_fields = ["id"]
    )

    results = long_term_memory_index.query(vector_query)
    logger.debug(f"Similar memory search results: {results}")

    if results:
        logger.debug(
            f"{len(results)} similar {'memory' if results.count == 1 else 'memories'} found. First: "
            f"{results[0]['id']}. Skipping storage."
        )
        return True
    
    return False

##### **b. Store long-term memories**

Below is a function that handles storing long-term memories in Redis with built-in deduplication.

It's a key part of our memory system that:
1. Prevents duplicate memories by checking for similar content
2. Creates vector embeddings for semantic search capabilities
3. Stores the memory with relevant metadata for future retrieval

We'll use the `similar_memory_exists()` function when we store memories in order to perform in-line memory deduplication.

In [17]:
from datetime import datetime
from typing import List, Optional, Union

import ulid

def store_memory(
    content: str,
    memory_type: MemoryType,
    user_id: str = SYSTEM_USER_ID,
    thread_id: Optional[str] = None,
    metadata: Optional[str] = None,
):
    """
    Store a long-term memory in Redis with deduplication

    This function:
    1. Checks for similar existing memories to avoid duplicates
    2. Generates vector embeddings for semantic search
    3. Stores the memory with metadata for retrieval
    """

    if metadata is None:
        metadata = "{}"

    logger.info(f"Preparing to store memory: {content}")

    if similar_memory_exists(content, memory_type, user_id, thread_id):
        logger.info("Similar memory found, skipping storage")
        return
    
    embedding = gg_embed.embed_documents([content])
    embedding = embedding[0]

    memory_data = {
        "user_id": user_id or SYSTEM_USER_ID,
        "content": content,
        "memory_type": memory_type.value,
        "metadata": metadata,
        "created_at": datetime.now().isoformat(),
        "embedding": embedding,
        "memory_id": str(ulid.ULID()),
        "thread_id": thread_id
    }

    try:
        long_term_memory_index.load([memory_data])
    except Exception as e:
        logger.error(f"Error storing memory: {e}")
        return
    
    logger.info(f"Stored {memory_type} memory: {content}")

##### **c. Retrieve relevant long-term memories**

And now that we're storing memories, we can retrieve them using vector similarity search with metadata filters using RedisVL.

This function: 
1. Takes a query string, optional filters (memory type, user ID, thread ID), and a distance threshold (semantic)
2. Creates a vector range query using the query's embedding
3. Builds a filter object based on passed options
4. Filters to narrow down the search results
5. Executes the search and returns parsed memory objects

In [18]:
def retrieve_memories(
    query: str,
    memory_type: Union[Optional[MemoryType], List[MemoryType]] = None,
    user_id: str = SYSTEM_USER_ID,
    thread_id: Optional[str] = None,
    distance_threshold: float = 0.1,
    limit: int = 5
) -> List[StoredMemory]:
    """
    Retrieve relevant memories from Redis using vector similarity search.
    """

    logger.debug(f"Retrieving memories for query: {query}")
    vector_query = VectorRangeQuery(
        vector = gg_embed.embed_query(query),
        return_fields = [
            "content",
            "memory_type",
            "metadata",
            "created_at",
            "memory_id",
            "thread_id",
            "user_id"
        ],
        num_results = limit,
        vector_field_name = "embedding",
        dialect = 2,
        distance_threshold = distance_threshold
    )

    base_filters = [f"@user_id:{{{user_id or SYSTEM_USER_ID}}}"]

    if memory_type:
        if isinstance(memory_type, list):
            base_filters.append(f"@memory_type:{{{'|'.join(memory_type)}}}")
        else:
            base_filters.append(f"@memory_type:{{{memory_type.value}}}")
    
    if thread_id:
        base_filters.append(f"@thread_id:{{{thread_id}}}")

    vector_query.set_filter(" ".join(base_filters))

    # Execute search
    results = long_term_memory_index.query(vector_query)

    # Parse results into StoredMemory objects
    memories = []
    for doc in results:
        try:
            memory = StoredMemory(
                id = doc["id"],
                memory_id = doc["memory_id"],
                user_id = doc["user_id"],
                thread_id = doc.get("thread_id", None),
                memory_type = MemoryType(doc["memory_type"]),
                content = doc["content"],
                created_at = doc["created_at"],
                metadata = doc["metadata"]
            )
            memories.append(memory)

        except Exception as e:
            logger.error(f"Error parsing memory: {e}")
            continue

    return memories

### **VI. Managing Long-Term Memory with Tools**

Memory operations are exposed as tools that the LLM can call to store or retrieve memories.

**Tool-based memory management:**
- LLM decided when to store/retrieve memories
- Fewer Redis calls but may miss some context
- Adds some latency due to LLM decision-making

Alternatively, you can always manually manage memories in your workflows.

**Manual memory management:**
- More Redis calls but faster response times
- Extracts more memories, providing richer context
- Higher token usage due to more context

=> NOTE: This tutorial uses tool-based memory for optimal balance of control and efficiency.

<img src="memory-agents.png" alt="alt text" width="500px" height="400px" style="background-color: lightblue;"/>

#### **1. Define Agent Tools**

Now that we have our storage functions defined, we can create the tools that will enable our agent to interact with the memory system. These tools will be used by the LLM to manage memories during conversations.

Let's start with the Store Memory Tool:

##### **a. Store Memory Tool**

This tool enables the agent to save important information as long-term memories in Redis. It's particularly useful for capturing:
- User preferences and habits
- Personal experiences and anecdotes
- Important facts and knowledge shared during conversations

The tool accepts the following parameters:
- `content`: The actual memory content to store (e.g., "User prefers window seats on flights")
- `memory_type`: The type of memory (e.g., `MemoryType.EPISODIC` for personal experiences, `MemoryType.SEMANTIC` for general knowledge)
- `metadata`: Optional dictionary for additional context (e.g., timestamps, source, confidence)
- `config`: Optional configuration for user/thread context (automatically handled by the agent)

When called, the tool:
1. Validates the input parameters
2. Stores the memory in Redis with proper indexing
3. Returns a success message with the stored content
4. Handles errors gracefully with informative messages

This tool is designed to be used by the LLM to build a persistent memory of the user's preferences and experiences, enabling more personalized and context-aware interactions overtime.

In [19]:
from typing import Dict, Optional

from langchain_core.tools import tool
from langchain_core.runnables.config import RunnableConfig

@tool
def store_memory_tool(
    content: str,
    memory_type: MemoryType,
    metadata: Optional[Dict[str, str]] = None,
    config: Optional[RunnableConfig] = None
) -> str:
    """
    Store a long-term memory in the system.

    Use this tool to save important information about user preferences,
    experiences, or general knowledge that might be useful in future
    interactions.
    """
    config = config or RunnableConfig()
    user_id = config.get("user_id", SYSTEM_USER_ID)
    thread_id = config.get("thread_id")

    try:
        store_memory(
            content = content,
            memory_type = memory_type,
            user_id = user_id,
            thread_id = thread_id,
            metadata = str(metadata) if metadata else None,
        )

        return f"Succesfully stored {memory_type} memory: {content}"
    except Exception as e:
        return f"Error storing memory: {str(e)}"

In [20]:
store_memory_tool.invoke({
    "content": "I like flying on Delta when possible", 
    "memory_type": "episodic"
})

15:53:10 __main__ INFO   Preparing to store memory: I like flying on Delta when possible
15:53:11 __main__ INFO   Stored MemoryType.EPISODIC memory: I like flying on Delta when possible


'Succesfully stored MemoryType.EPISODIC memory: I like flying on Delta when possible'

##### **b. Retrieve Memories Tool**
This tool allows us to search through our stored memories using semantic similarity and filtering.

This tool is particularly useful when you want to:
- Find relevant past experiences or preferences
- Filter memories by type (episodic or semantic)
- Get user-specific information
- Limit the number of results to keep responses focused

The tool works by:
1. Taking a query string and searching for semantically similar memories
2. Filtering results based on memory type
3. Applying a similarity threshold to ensure relevance
4. Formatting the results in a clear, readable way

In [21]:
@tool
def retrieve_memories_tool(
    query: str,
    memory_type: List[MemoryType],
    limit: int = 5,
    config: Optional[RunnableConfig] = None
) -> str:
    """
    Retrieve long-term memories relevant to the query.

    Use this tool to access previously stored information about user
    preferences, experiences, or general knowledge.
    """
    config = config or RunnableConfig()
    user_id = config.get("user_id", SYSTEM_USER_ID)

    try:
        stored_memories = retrieve_memories(
            query = query,
            memory_type = memory_type,
            user_id = user_id,
            limit = limit,
            distance_threshold = 0.3
        )

        response = []
        if stored_memories:
            response.append("Long-term memories:")
            for memory in stored_memories:
                response.append(f"- [{memory.memory_type}] {memory.content}")

        return "\n".join(response) if response else "No relevant memories found."
    except Exception as e:
        return f"Error retrieving memories: {str(e)}"

In [22]:
retrieve_memories_tool.invoke({
    "query": "Airline preferences", 
    "memory_type": ["episodic"]
})

'Long-term memories:\n- [MemoryType.EPISODIC] I like flying on Delta when possible'

### **VII. Build the Travel Agent**

#### **1. Setting up the ReAct Agent**

We'll use LangGraph's prebuilt components to create a ReAct agent with memory capabilities:
1. **Short-term Memory**: A checkpoint saver tracks conversation history per thread
2. **Long-term Memory**: We'll extract and store key information from conversations
- Episodic memories: User preferences and experiences
- Semantic memories: General travel knowledge

The system will automatically summarize conversations to manage context while preserving important details in long-term storage.

Below we start with setting up the Redis checkpointer (`RedisSaver`) that will handle short term memory for the agent.

In [23]:
from langchain_core.messages import AIMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.redis import RedisSaver

# Setup the Redis checkpointer for short term memory
redis_saver = RedisSaver(redis_client = redis_client)
redis_saver.setup()

16:01:19 langgraph.checkpoint.redis INFO   Redis client is a standalone client


In [24]:
# Define set of tools
tools = [store_memory_tool, retrieve_memories_tool]

In [25]:
# Configure an LLM for the agent with a more creative temperature
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.7
).bind_tools(tools)

In [26]:
system_prompt = SystemMessage(
    content = """
    You are a travel assistant helping users plan their trips. You remember user preferences
    and provide personalized recommendations based on past interactions.

    You have access to the following types of memory:
    1. Short-term memory: The current conversation thread
    2. Long-term memory:
        - Episodic: User preferences and past trip experiences (e.g., "User prefers window seats")
        - Semantic: General knowledge about travel destinations and requirements

    Your procedural knowledge (how to search, book flights, etc.) is built into your tools and prompts.

    Always be helpful, personal, and context-aware in your responses.
    """
)

# Define the travel agent
travel_agent = create_react_agent(
    model = llm, 
    tools = tools,
    checkpointer = redis_saver,
    prompt = system_prompt
)

#### **2. Build nodes of LangGraph workflow**

##### **a. Node 1: Respond to the user**

In LangGraph, a node represents a discrete unit of processing in a workflow. Each node is a function that takes a state object and configuration as input, processes the data, and returns an updated state. Nodes can be connected to from a directed graph that defines the flow of execution.

The `respond_to_user` node is our first node in the travel agent workflow. It serves as the entry point for user interactions and handles the core conversation flow.
Here's how it works:
1. It receives the current conversation state and configuration
2. Extracts any human messages from the state
3. Invokes our travel agent to generate a repsonse
4. Handles any errors gracefully
5. Updates the conversation state with the agent's response

The node uses a custom `RuntimeState` class that inherits from `MessagesState` to maintain the conversation history. This state object is passed between nodes in the graph, allowing each node to access and modify the conversation context as needed.

In [28]:
from langgraph.graph.message import MessagesState

class RuntimeState(MessagesState):
    """Runtime state for the travel agent."""
    pass

In [29]:
from langchain_core.messages import HumanMessage

def respond_to_user(
    state: RuntimeState,
    config: RunnableConfig
) -> RuntimeState:
    """Invoke the travel agent to generate a response."""
    human_messages = [m for m in state["messages"]
                      if isinstance(m, HumanMessage)]
    
    if not human_messages:
        logger.warning("No HumanMessage found in state")
        return state
    
    try:
        # Single agent invocation, not streamed (simplified for reliablity)
        result = travel_agent.invoke(
            input = {"messages": state["messages"]},
            config = config
        )
        agent_message = result["messages"][-1]
        state["messages"].append(agent_message)
    except Exception as e:
        logger.error(f"Error invoking travel agent: {e}")
        agent_message = AIMessage(
            content = "I'm sorry, I encountered an error processing your request."
        )
        state["messages"].append(agent_message)
    
    return state

##### **b. Node 2: Execute Tools**

The `execute_tools` node is a critical component in our travel agent's workflow that bridges the gap between the LLM's decisions and actual tool execution. Positioned after the `respond_to_user` node, it handles the pratical side of the agent's tool-using capabilities.

When the LLm determines it needs to use a tool, it includes tool calls in its response. This node then:
1. Scans the conversation history to find the most recent AI message containing tool calls
2. For each tool call found:
- Extracts the tool name, arguments and call ID from message
- Matches the tool name against our available tools
- Executes the tool with the provided arguments
- Creates a ToolMessage containing the result
3. Handles any errors that occur during tool execution
4. Adds all tool results back to the conversation history

This node is essential because it enables our agent to interact with external systems and services while maintaining a coherent conversation flow. Without it, the agent would be limited to just generating text responses without the ability to perform actual actions or retrieve real-time information

In [30]:
from langchain_core.messages import ToolMessage

def execute_tools(
    state: RuntimeState,
    config: RunnableConfig
) -> RuntimeState:
    """Execute tools specified in the latest AIMessage and append ToolMessages."""
    messages = state["messages"]
    latest_ai_message = next(
        (
            m for m in reversed(messages) 
            if isinstance(m, AIMessage) and m.tool_calls
        ),
        None
    )

    if not latest_ai_message:
        return state # No tool calls to process
    
    tool_messages = []
    for tool_call in latest_ai_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]

        # Find the corresponding tool
        tool = next((t for t in tools if t.name == tool_name), None)
        if not tool:
            continue # Skip if tool not found

        try:
            # Execute the tool with the provided arguments
            result = tool.invoke(tool_args, config=config)

            # Create a ToolMessage with the result
            tool_message = ToolMessage(
                content = str(result),
                tool_call_id = tool_id,
                name = tool_name
            )
            tool_messages.append(tool_message)
        except Exception as e:
            error_message = ToolMessage(
                content = f"Error executing tool '{tool_name}': {str(e)}",
                tool_call_id = tool_id,
                name = tool_name
            )
            tool_messages.append(error_message)

        # Append the ToolMessages to the message history
        messages.extend(tool_messages)
        state["messages"] = messages
        return state

##### **c. Node 3: Conversation Summarization**

The conversation summarization node helps manage context by condensing chat history into concise summaries. This prevents the LLM's context window from being overwhelmed as the conversation grows.

Key features:
1. **Automatic Triggering**: Summarizes after every 6 messages (configurable)
2. **Smart Summarization**:
- Uses `gemini-2.5-flash` with low temperature (0.3) for consistent summaries
- Preserves key information like preferences and pending items
- Replaces old messages while keeping recent context
3. **Benefits**:
- Prevents context window overflow
- Maintains conversation coherence
- Optimizes token usage

The summary becomes part of the conversation history allow the agent to reference past interactions efficiently.

In [31]:
from langchain_core.messages import RemoveMessage

# An LLM configured for summarization
summarizer = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature = 0.3)

# The number of messages after which we'll summarize the conversation
MESSAGE_SUMMARIZATION_THRESHOLD = 6


def summarize_conversation(
    state: RuntimeState,
    config: RunnableConfig
) -> RuntimeState:
    """
    Summarize a list of messages into a concise summary to reduce context length
    while preserving important information.
    """

    messages = state["messages"]
    current_message_count = len(messages)
    if current_message_count < MESSAGE_SUMMARIZATION_THRESHOLD:
        logger.debug(f"Not summarizing conversation: {current_message_count}")
        return state

    system_prompt = """
    You are a conversation summarizer. Create a concise summary of the previous
    conversation between a user and a travel assistant.

    The summary should:
    1. Highlight key topics, preferences, and decisions
    2. Include any specific trip details (destinations, dates, preferences)
    3. Note any outstanding questions or topics that need follow-up
    4. Be concise but informative

    Format your summary as a brief narrative paragraph.
    """

    message_content = '\n'.join(
        [
            f"{'User' if isinstance(msg, HumanMessage) else 'Assistant'}: {msg.content}"
            for msg in messages
        ]
    )

    summary_messages = [
        SystemMessage(content = system_prompt),
        HumanMessage(
            content = f"Please summarize this conversation:\n\n{message_content}"
        ),
    ]

    summary_response = summarizer.invoke(summary_messages)

    logger.info(f"Summarized {len(messages)} messages into a conversation summary")

    summary_message = SystemMessage(
        content = f"""
        Summary of the conversation so fat:

        {summary_response.content}

        Please continue the conversation based on this summary and the recent messages.
        """
    )

    remove_messages = [
        RemoveMessage(id = msg.id) for msg in messages if msg.id is not None
    ]

    state["messages"] = [
        *remove_messages,
        summary_message,
        state["messages"][-1]
    ]

    return state.copy()

#### **3. Assemble the full graph**

It's time to assemble our graph for end-to-end agent execution. We will attach all 3 nodes we defined above.

In [35]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(RuntimeState)

# Add nodes to the graph
workflow.add_node("agent", respond_to_user)
workflow.add_node("execute_tools", execute_tools)
workflow.add_node("summarize_conversation", summarize_conversation)

Next, we''ll tie the nodes together using edges which control process flow. There's a conditional edge between the agent node and what comes next. What comes next is based on whether we need to handle + execute a tool call or proceed

In [36]:
def decide_next_step(state):
    latest_ai_message = next(
        (
            m for m in reversed(state["messages"])
            if isinstance(m, AIMessage)
        ),
        None
    )
    if latest_ai_message and latest_ai_message.tool_calls:
        return "execute_tools"
    return "summarize_conversation"

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    decide_next_step,
    {
        "execute_tools": "execute_tools",
        "summarize_conversation": "summarize_conversation"
    }
)
workflow.add_edge("execute_tools", "agent")
workflow.add_edge("summarize_conversation", END)

In [37]:
# Compiled the graph
graph = workflow.compile(checkpointer=redis_saver)

### **VII. Testing the Main Agent Loop**

Let's put our travel agent to work! The main loop handles the conversation flow:
1. **Initialization**: Sets up a unique thread ID and empty message state
2. **Input Processing**: Gets user input, handles empty inputs and exit commands
3. **Message Flow**: Converts input to HumanMessage and streams through our workflow
4. **Reponse Generation**: Processes state and displays AI responses
5. **Error Handling**: Catches errors and keeps the conversation flowing smoothly

In [38]:
def main(
    thread_id: str = "book_flight",
    user_id: str = "demo_user"
):
    """Main interaction loop for the travel agent"""
    print("Welcome to the Travel Assistant! (Type 'exit' to quit)")

    config = RunnableConfig(
        configurable = {
            "thread_id": thread_id,
            "user_id": user_id
        }
    )
    state = RuntimeState(messages = [])

    while True:
        user_input = input("\nYou (type 'quit' to quit): ")

        if not user_input:
            continue

        if user_input.lower() in ["exit", "quit"]:
            print("Thank you for using the Travel Assistant. Goodbye!")
            break

        state["messages"].append(HumanMessage(content=user_input))

        try:
            # Process user input through the graph
            for result in graph.stream(state, config=config, stream_mode="values"):
                state = RuntimeState(**result)

            logger.debug(f"# of messages after run: {len(state["messages"])}")

            # Find the most recent AI message, so we can print the repsonse
            ai_messages = [m for m in state["messages"] if isinstance(m, AIMessage)]
            if ai_messages:
                message = ai_messages[-1].content
            else:
                logger.error("No AI messages after run")
                message = "I'm sorry, I couldn't process your request properly."

                # Add the error message to the state
                state["messages"].append(AIMessage(content=message))

            print(f"\nAssistant: {message}")
        except Exception as e:
            logger.exception(f"Error processing request: {e}")
            error_message = "I'm sorry, I encountered an error processing your request."
            print(f"\nAssistant: {error_message}")
            # Add the error message to the state
            state["messages"].append(AIMessage(content=error_message))

Before you try your own, take a look at the current conversation between Tyler and the travel agent. Notice the memory storage actions, the calls to the LLM, and also the conversation summarization that take plact during the workflow!

In [39]:
try:
    user_id = input("Enter a user ID: ") or "demo_user"
    thread_id = input("Enter a thread ID: ") or "demo_thread"
except Exception as e:
    exit()
else:
    main(thread_id, user_id)

Welcome to the Travel Assistant! (Type 'exit' to quit)

Assistant: That sounds like a fantastic trip! Singapore is an amazing choice for both outdoor activities and incredible food experiences.

For **outdoors**, you absolutely have to visit:

*   **Gardens by the Bay**: Explore the Supertree Grove, Cloud Forest, and Flower Dome. It's a stunning futuristic park that's beautiful both day and night.
*   **Singapore Botanic Gardens**: A UNESCO World Heritage Site, perfect for a leisurely stroll amidst lush greenery, including the National Orchid Garden.
*   **MacRitchie Reservoir Park**: If you're up for a bit more adventure, you can go for a hike, including the TreeTop Walk, which offers a panoramic view of the forest canopy.

And for **food**, you're in for a treat! Singapore is a culinary paradise:

*   **Hawker Centers**: This is where you'll find the heart of Singaporean cuisine. I highly recommend trying **Lau Pa Sat** for its vibrant atmosphere and satay street at night, and **Maxw

In [42]:
res = retrieve_memories_tool.invoke({"query": "", "memory_type": ["episodic", "semantic"]})
res.split("\n")

['Long-term memories:',
 '- [MemoryType.EPISODIC] I like flying on Delta when possible']

In [41]:
from redisvl.query import CountQuery

# count total long-term memories in Redis
long_term_memory_index.query(CountQuery())

3